# Reporte de promesas acumuladas en el mes

In [1]:
import pandas as pd
import numpy as np
import mysql.connector
from mysql.connector import Error
from datetime import date
from datetime import datetime
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
#Definimos la ruta en donde queremos guardar los archivos
ruta = '/home/estadistico/Documents/Erick/Reportes diarios/Promesas Mes'
#Definimos la ruta donde estan las asignaciones
ruta_asig_baz = '/home/estadistico/Documents/Erick/Banco Azteca/Asignacion csv'
mes = 20201201
mes2 = 'Diciembre2020'

In [3]:
#Agregamos las variables a ocupar
servidor = '192.168.15.12'
puerto = int('3306')
usuario = 'estadisticas'
contraseña = 'estadisticas8474'
base = 'procesos_externos'

In [4]:
#Asignamos valores a los parametros \n",
today = date.today().strftime('%Y%m%d')
now = datetime.now().strftime('%d-%m-%Y-%H:%M')

In [5]:
#Hacemos la conexion con el servidor\n",
try:
    conn = mysql.connector.connect(user=usuario,
                               password=contraseña,
                               host=servidor,
                               port=3306,
                               database=base)
    conn.set_charset_collation('latin1')
except mysql.Error as e:
    print("Failed to execute stored procedure: {}".format(error))

cursor = conn.cursor()

# Liverpool

In [6]:
#Hacemos la consulta referente a las gestiones de Liverpool
sql_gest_liv = cursor.callproc('liverpool_rpt_gestiones_detallado', [mes,today])
for result in cursor.stored_results():
    gestion_liv = pd.DataFrame(result.fetchall())
gestion_liv.columns = ['folio_gestion','firma_id','unegocio_id','credito','nombre_credito','telefono','tipo_telefono','fecha_gestion','usuario','nombre_usuario','dictamen','accion','resultado','fecha_promesa','monto_promesa','comentarios']
gestion_liv = gestion_liv.loc[gestion_liv['dictamen']=='PROMESA']
gestion_liv['credito'] = pd.to_numeric(gestion_liv['credito'])
#gestion_liv.head(3)

In [7]:
#Hacemos la consulta de la asignacion
sql_asig_liv = cursor.callproc('liverpool_rpt_asignacion_activa',)
for result in cursor.stored_results():
    asignacion_liv = pd.DataFrame(result.fetchall())
asignacion_liv = asignacion_liv.iloc[:,[0,1,3,24,6,26]]
asignacion_liv.columns = ['firma_id','unegocio_id','credito','estado','rfc','division']
asignacion_liv['credito'] = pd.to_numeric(asignacion_liv['credito'])
#asignacion_liv.head(3)

In [8]:
#Hacemos el consolidado 
consolidado_liv = pd.merge(gestion_liv,asignacion_liv,how='left',on=['firma_id','unegocio_id','credito'])
cols = ['monto_promesa','credito']
consolidado_liv[cols] = consolidado_liv[cols].apply(pd.to_numeric, errors='coerce')
consolidado_liv = consolidado_liv[(consolidado_liv['monto_promesa']>50)]
consolidado_liv = consolidado_liv[(consolidado_liv['monto_promesa']<500000)]
consolidado_liv.fillna('COBRANZA',inplace=True)
#consolidado_liv.head(3)

In [9]:
#Obtenemos datos de las promesas de Liverpool
consolidado_liv2 = consolidado_liv.drop_duplicates(subset=['credito','fecha_promesa','monto_promesa'])
pivot_liv = pd.pivot_table(consolidado_liv2,index=['division'],values=['credito','monto_promesa'],aggfunc=['count',np.sum],margins=True)
pivot_liv = pd.DataFrame(pivot_liv.to_records())
pivot_liv = pivot_liv.iloc[:,[0,1,4]]
pivot_liv.columns = ['Division','NumeroPromesas','SumaPromesas']
pivot_liv.fillna(0,inplace=True)
#pivot_liv[['NumeroPromesas','PromedioPromesa','SumaPromesas']] = pivot_liv[['NumeroPromesas','PromedioPromesa','SumaPromesas']].applymap("{0:.2f}".format)
pivot_liv

,Division,NumeroPromesas,SumaPromesas
0,,2,4000.00
1,COBRANZA,6,13627.00
2,REM2,40,186946.81
3,REM4,23,103388.14
4,REM9,18,27963.00
5,SB02,64,105473.65
6,pren,34,104692.00
7,All,187,546090.60


# Bradesco 

In [10]:
#Hacemos la consulta referente a las gestiones de Bradesco
sql_gest_brad = cursor.callproc('bradescard_rpt_gestiones_detallado', [mes,today])
for result in cursor.stored_results():
    gestion_brad = pd.DataFrame(result.fetchall())
gestion_brad.columns = ['folio_gestion','firma_id','unegocio_id','credito','nombre_credito','telefono','tipo_telefono','fecha_gestion','usuario','nombre_usuario','dictamen','accion','resultado','accion_resultado','fecha_promesa','monto_promesa','comentarios']
gestion_brad = gestion_brad.loc[gestion_brad['dictamen']=='PROMESA']
gestion_brad['credito'] = pd.to_numeric(gestion_brad['credito'])
gestion_brad['monto_promesa'] = pd.to_numeric(gestion_brad['monto_promesa'])
#gestion_brad.head(3)

In [22]:
#Hacemos la consulta de la asignacion
sql_asig_brad = cursor.callproc('bradescard_rpt_asignacion_activa',)
for result in cursor.stored_results():
    asignacion_brad = pd.DataFrame(result.fetchall())
asignacion_brad = asignacion_brad.iloc[:,[0,1,3,24,6,26]]
asignacion_brad.columns = ['firma_id','unegocio_id','credito','estado','rfc','division']
asignacion_brad['credito'] = pd.to_numeric(asignacion_brad['credito'])
asignacion_brad.head(3)

,firma_id,unegocio_id,credito,estado,rfc,division
0,1,3,2869000138080177,YUCATAN,JURM8405047J801,C0 5PV
1,1,3,2869000240445953,VERACRUZ,TOHI650826EZ601,C0 4PV
2,1,3,2869000320452929,PUEBLA,HERL821006J1401,C0 4PV


In [23]:
#Hacemos la union de asignacion y promesas
consolidado_brad = pd.merge(gestion_brad,asignacion_brad,how='left',on=['firma_id','unegocio_id','credito'])
cols = ['monto_promesa','credito']
consolidado_liv[cols] = consolidado_liv[cols].apply(pd.to_numeric, errors='coerce')
consolidado_brad = consolidado_brad[(consolidado_brad['monto_promesa']>50)]
consolidado_brad = consolidado_brad[(consolidado_brad['monto_promesa']<500000)]
consolidado_brad.fillna('COBRANZA',inplace=True)
#consolidado_brad.head(3)

In [24]:
#Obtenemos datos de las promesas de Bradesco
consolidado_brad2 = consolidado_brad.drop_duplicates(subset=['credito','fecha_promesa','monto_promesa'])
pivot_brad = pd.pivot_table(consolidado_brad2,index=['division'],values=['credito','monto_promesa'],aggfunc=['count',np.sum],margins=True)
pivot_brad = pd.DataFrame(pivot_brad.to_records())
pivot_brad = pivot_brad.iloc[:,[0,1,4]]
pivot_brad.columns = ['Division','NumeroPromesas','SumaPromesas']
pivot_brad.fillna(0,inplace=True)
#pivot_brad[['NumeroPromesas','PromedioPromesa','SumaPromesas']] = pivot_gestion_brad[['NumeroPromesas','PromedioPromesa','SumaPromesas']].applymap("{0:.2f}".format)
pivot_brad

,Division,NumeroPromesas,SumaPromesas
0,A,3,3400.0
1,B,5,3650.0
2,C0 4PV,124,231715.0
3,C0 5PV,49,72150.0
4,C0 6PV,53,101713.0
5,C1,108,159102.0
6,C2,45,70602.0
7,C3,77,88709.0
8,C4,20,26853.0
9,All,484,757894.0


# Credifiel

In [25]:
#Hacemos la consulta referente a las gestiones de Credifiel
sql_gest_cred = cursor.callproc('credifiel_rpt_gestiones_detallado', [mes,today])
for result in cursor.stored_results():
    gestion_cred = pd.DataFrame(result.fetchall())
gestion_cred.columns = ['folio_gestion','unegocio_id','fecha_gestion','hora_gestion','credito','nombre_credito','telefono','usuario','nombre_usuario','accion','resultado','fecha_promesa','monto_promesa','comentarios','accion_credifiel','resultado_credifiel','dictamen']
gestion_cred['monto_promesa'] = pd.to_numeric(gestion_cred['monto_promesa'])
gestion_cred = gestion_cred.loc[gestion_cred['accion_credifiel']=='PDP']
gestion_cred.head(3)

,folio_gestion,unegocio_id,fecha_gestion,hora_gestion,credito,nombre_credito,telefono,usuario,nombre_usuario,accion,resultado,fecha_promesa,monto_promesa,comentarios,accion_credifiel,resultado_credifiel,dictamen
6,9229469,1,2020-12-01,07:29:39,CR-267152,LIEVANO CAL Y MAYOR VERONICA,9616189250,HERNANDEZGARO,HERNANDEZ GALINDO RODRIGO,LLAMADA A TITULAR,PROMESA DE PAGO PARCIAL,2020-12-15,4294.00,9616189250 TT REALIZARA PAGO 6 $4294 15/12/20,PDP,Promesa de Pago,CONTACTO
7,9229522,1,2020-12-01,07:34:46,CR-494364,DE LA VEGA ROMERO MIGUEL ANGEL,5959250197,HERNANDEZGARO,HERNANDEZ GALINDO RODRIGO,LLAMADA A TITULAR,PROMESA DE PAGO PARCIAL,2020-12-15,3148.83,5959250197 SE GENERA PAGO 9 POR $3148.83 PARA ...,PDP,Promesa de Pago,CONTACTO
11,9229759,1,2020-12-01,07:49:21,CR-503789,MARTINEZ MANRIQUE LUCY,5588471578,HERNANDEZGARO,HERNANDEZ GALINDO RODRIGO,LLAMADA A TITULAR,PROMESA DE PAGO PARCIAL,2020-12-01,2000.00,5588471578 TT REALIZARA LIQUIDACION CON 32000...,PDP,Promesa de Pago,CONTACTO


In [26]:
#Hacemos la consulta de la asignacion
sql_asig_cred = cursor.callproc('credifiel_rpt_asignacion_activa',)
for result in cursor.stored_results():
    asignacion_cred = pd.DataFrame(result.fetchall())
asignacion_cred = asignacion_cred.iloc[:,[1,3,32]]
asignacion_cred.columns = ['unegocio_id','credito','division']
#asignacion_brad['credito'] = pd.to_numeric(asignacion_brad['credito'])
asignacion_cred.head(3)

,unegocio_id,credito,division
0,1,CR-101922,COBRANZA EXTRAJUDICIAL
1,1,CR-101949,LIQUIDAR
2,1,CR-101952,LIQUIDAR


In [27]:
#Hacemos el consolidado de la asignacion y promesado
consolidado_cred = pd.merge(gestion_cred,asignacion_cred,how='left',on=['unegocio_id','credito'])
cols = ['monto_promesa','unegocio_id']
consolidado_cred[cols] = consolidado_cred[cols].apply(pd.to_numeric, errors='coerce')
consolidado_cred = consolidado_cred.loc[consolidado_cred['monto_promesa']>50]
consolidado_cred = consolidado_cred[(consolidado_cred['monto_promesa']<500000)]
consolidado_cred.fillna('COBRANZA',inplace=True)
consolidado_cred.replace('','COBRANZA',inplace=True)
consolidado_cred.head(3)

,folio_gestion,unegocio_id,fecha_gestion,hora_gestion,credito,nombre_credito,telefono,usuario,nombre_usuario,accion,resultado,fecha_promesa,monto_promesa,comentarios,accion_credifiel,resultado_credifiel,dictamen,division
0,9229469,1,2020-12-01,07:29:39,CR-267152,LIEVANO CAL Y MAYOR VERONICA,9616189250,HERNANDEZGARO,HERNANDEZ GALINDO RODRIGO,LLAMADA A TITULAR,PROMESA DE PAGO PARCIAL,2020-12-15,4294.00,9616189250 TT REALIZARA PAGO 6 $4294 15/12/20,PDP,Promesa de Pago,CONTACTO,EXTRAJUDICIAL
1,9229522,1,2020-12-01,07:34:46,CR-494364,DE LA VEGA ROMERO MIGUEL ANGEL,5959250197,HERNANDEZGARO,HERNANDEZ GALINDO RODRIGO,LLAMADA A TITULAR,PROMESA DE PAGO PARCIAL,2020-12-15,3148.83,5959250197 SE GENERA PAGO 9 POR $3148.83 PARA ...,PDP,Promesa de Pago,CONTACTO,COBRANZA EXTRAJUDICIAL
2,9229759,1,2020-12-01,07:49:21,CR-503789,MARTINEZ MANRIQUE LUCY,5588471578,HERNANDEZGARO,HERNANDEZ GALINDO RODRIGO,LLAMADA A TITULAR,PROMESA DE PAGO PARCIAL,2020-12-01,2000.00,5588471578 TT REALIZARA LIQUIDACION CON 32000...,PDP,Promesa de Pago,CONTACTO,COBRANZA EXTRAJUDICIAL


In [28]:
#Obtenemos datos de las promesas de Credifiel
consolidado_cred2 = consolidado_cred.drop_duplicates(subset=['credito','fecha_promesa','monto_promesa'])
pivot_cred = pd.pivot_table(consolidado_cred,index=['division'],values=['unegocio_id','monto_promesa'],aggfunc=['count',np.sum],margins=True)
pivot_cred = pd.DataFrame(pivot_cred.to_records())
pivot_cred = pivot_cred.iloc[:,[0,2,3]]
pivot_cred.columns = ['Accion_Credifiel','NumeroPromesas','SumaPromesas']
pivot_cred.fillna(0,inplace=True)
#pivot_gestion_cred[['NumeroPromesas','PromedioPromesa','SumaPromesas']] = pivot_gestion_brad[['NumeroPromesas','PromedioPromesa','SumaPromesas']].applymap("{0:.2f}".format)
pivot_cred

,Accion_Credifiel,NumeroPromesas,SumaPromesas
0,COBRANZA,5,6832.93
1,COBRANZA EXTRAJUDICIAL,20,88756.34
2,EXTRAJUDICIAL,4,13411.72
3,All,29,109000.99


# Banco Azteca

In [29]:
#Hacemos la consulta referente a las gestiones de Banco Azteca
sql_gest_baz = cursor.callproc('baz_rpt_gestiones_detallado', [mes,today])
for result in cursor.stored_results():
     gestion_baz = pd.DataFrame(result.fetchall())
gestion_baz.columns = ['folio_gestion','firma_id','unegocio_id','credito','nombre_credito','telefono','tipo_telefono','fecha_gestion','usuario','nombre_usuario','dictamen','accion_resultado','fecha_promesa','monto_promesa','comentarios']
gestion_baz = gestion_baz.loc[gestion_baz['dictamen']=='PROMESA']
gestion_baz.head(3)

,folio_gestion,firma_id,unegocio_id,credito,nombre_credito,telefono,tipo_telefono,fecha_gestion,usuario,nombre_usuario,dictamen,accion_resultado,fecha_promesa,monto_promesa,comentarios
118,9286847,7,2,1-1-4221-45904,EMELI ESMERALDA RAMIREZ ACEVEDO,3315276940,celular,2020-12-05 13:32:28,LEONSASU,LEON SALINAS SUSANA,PROMESA,ENTRADA WHATSAPP-PROMESA DE PAGO PARCIAL,2020-12-12,604.00,3315276940 MAN SE CONTACTA TT VIA WHAT PARA R...
185,9286556,7,2,1-1-151-42488,JOSE ALFREDO CORONA MENDOZA,4444067499,celular,2020-12-05 13:19:41,CARDOSASEVEBRE,BRENDA EVELYN CARDOSA SEVEROÂ,PROMESA,GESTION TELEFONICA -PROMESA DE PAGO PARCIAL,2020-12-07,3000.00,4444067499 4444067499 SE AGENDA PAGO DE 3000...
349,9285838,7,2,1-3-9098-14378,ADELA LOPEZ NAVARRO,4444920669,celular,2020-12-05 12:46:08,CARDOSASEVEBRE,BRENDA EVELYN CARDOSA SEVEROÂ,PROMESA,GESTION TELEFONICA -PROMESA DE PAGO PARCIAL,2020-12-12,2796.00,4444920669 4444920669 se confirma pago y se ag...


In [30]:
asignacion_baz = pd.read_csv(''+ruta_asig_baz+'/Asignacion banco azteca '+str(mes2)+'.csv')
asignacion_baz = asignacion_baz.rename(columns=str.lower)
asignacion_baz.head(3)

,#fiidcampana,fipais,ficanal,fisucursal,fifolio
0,164,1,1,5496,4183
1,164,1,1,110,52389
2,164,1,1,5254,28682


In [31]:
asignacion_baz['credito'] = asignacion_baz['fipais'].map(str)+'-'+asignacion_baz['ficanal'].map(str)+'-'+asignacion_baz['fisucursal'].map(str)+'-'+asignacion_baz['fifolio'].map(str)
asignacion_baz.head(3)

,#fiidcampana,fipais,ficanal,fisucursal,fifolio,credito
0,164,1,1,5496,4183,1-1-5496-4183
1,164,1,1,110,52389,1-1-110-52389
2,164,1,1,5254,28682,1-1-5254-28682


In [32]:
consolidado_baz = pd.merge(gestion_baz,asignacion_baz,how='left',on='credito')
cols = ['monto_promesa','#fiidcampana']
consolidado_baz[cols] = consolidado_baz[cols].apply(pd.to_numeric, errors='coerce')
consolidado_baz = consolidado_baz[(consolidado_baz['monto_promesa']>50)]
consolidado_baz = consolidado_baz[(consolidado_baz['monto_promesa']<500000)]
consolidado_baz.fillna('COBRANZA',inplace=True)
consolidado_baz.head(3)

,folio_gestion,firma_id,unegocio_id,credito,nombre_credito,telefono,tipo_telefono,fecha_gestion,usuario,nombre_usuario,dictamen,accion_resultado,fecha_promesa,monto_promesa,comentarios,#fiidcampana,fipais,ficanal,fisucursal,fifolio
0,9286847,7,2,1-1-4221-45904,EMELI ESMERALDA RAMIREZ ACEVEDO,3315276940,celular,2020-12-05 13:32:28,LEONSASU,LEON SALINAS SUSANA,PROMESA,ENTRADA WHATSAPP-PROMESA DE PAGO PARCIAL,2020-12-12,604.0,3315276940 MAN SE CONTACTA TT VIA WHAT PARA R...,COBRANZA,COBRANZA,COBRANZA,COBRANZA,COBRANZA
1,9286556,7,2,1-1-151-42488,JOSE ALFREDO CORONA MENDOZA,4444067499,celular,2020-12-05 13:19:41,CARDOSASEVEBRE,BRENDA EVELYN CARDOSA SEVEROÂ,PROMESA,GESTION TELEFONICA -PROMESA DE PAGO PARCIAL,2020-12-07,3000.0,4444067499 4444067499 SE AGENDA PAGO DE 3000...,COBRANZA,COBRANZA,COBRANZA,COBRANZA,COBRANZA
2,9285838,7,2,1-3-9098-14378,ADELA LOPEZ NAVARRO,4444920669,celular,2020-12-05 12:46:08,CARDOSASEVEBRE,BRENDA EVELYN CARDOSA SEVEROÂ,PROMESA,GESTION TELEFONICA -PROMESA DE PAGO PARCIAL,2020-12-12,2796.0,4444920669 4444920669 se confirma pago y se ag...,COBRANZA,COBRANZA,COBRANZA,COBRANZA,COBRANZA


In [33]:
#Obtenemos datos de las promesas de Banco Azteca
consolidado_baz2 = consolidado_baz.drop_duplicates(subset=['credito','fecha_promesa','monto_promesa'])
pivot_baz = pd.pivot_table(consolidado_baz,index=['dictamen'],values=['unegocio_id','monto_promesa'],aggfunc=('count',np.sum))
pivot_baz = pd.DataFrame(pivot_baz.to_records())
pivot_baz = pivot_baz.iloc[:,[0,3,2]]
pivot_baz.columns = ['Dictamen','NumeroPromesas','SumaPromesas']
pivot_baz.fillna(0,inplace=True)
#pivot_baz[['NumeroPromesas','PromedioPromesa','SumaPromesas']] = pivot_gestion_baz[['NumeroPromesas','PromedioPromesa','SumaPromesas']].applymap("{0:.2f}".format)
pivot_baz

,Dictamen,NumeroPromesas,SumaPromesas
0,PROMESA,377,376955.5


In [34]:
writer = pd.ExcelWriter(''+ruta+'/Reporte Promesas Acumulado Mes '+now+'.xlsx', engine='xlsxwriter')
pivot_liv.to_excel(writer,'PromMesLiverpool',index=False,header=True)
pivot_brad.to_excel(writer,'PromMesBradesco',index=False,header=True)
pivot_cred.to_excel(writer,'PromMesCredifiel',index=False,header=True)
pivot_baz.to_excel(writer,'PromBancoAzteca',index=False,header=True)

writer.save()
writer.close()